In [2]:
import pandas as pd
from src.services.leads import get_leads, get_last_lead
from src.services.cases import get_many_cases
from collections.abc import MutableMapping
import datetime

## Data Loading

In [3]:
leads_not_found = get_last_lead(
    start_date=datetime.datetime(2023, 11, 15),
    end_date=datetime.datetime(2023, 11, 3),
    status='not_found',
    limit=500
)

/Users/aennassiri/Projects/Personal/ticket-washer/src/services/leads.py:104: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", ">=", start_date)
/Users/aennassiri/Projects/Personal/ticket-washer/src/services/leads.py:108: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("case_date", "<=", end_date)
/Users/aennassiri/Projects/Personal/ticket-washer/src/services/leads.py:110: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  leads_list = leads_list.where("status", "==", status)


In [4]:
len(leads_not_found)

TypeError: object of type 'NoneType' has no len()

In [5]:
# Get cases 30 by 30
from src.components.leads import lead


cases = []
for i in range(0, len(leads_not_found), 30):
    cases += get_many_cases(
        [lead.case_id for lead in leads_not_found[i : i + 30]]
    )

cases = [c for c in cases if c.address_state_code is not None]

TypeError: object of type 'NoneType' has no len()

In [60]:
cases = cases[:500]

In [61]:
leads_not_found = get_last_lead(
    start_date=datetime.datetime(2023, 10, 15),
    end_date=datetime.datetime(2023, 11, 1),
    status="contacted",
    limit=500,
)

In [62]:
for i in range(0, len(leads_not_found), 30):
    cases += get_many_cases(
        [lead.case_id for lead in leads_not_found[i : i + 30]]
    )

In [63]:
cases = [c for c in cases if c.address_state_code is not None]

In [64]:
cases = cases[:1001]

In [65]:
df = pd.DataFrame([case.model_dump() for case in cases])

In [66]:
df.columns

Index(['case_id', 'court_id', 'participants', 'related_cases',
       'protection_order', 'parties', 'disposed', 'legal_fileaccepted',
       'paper_accepted', 'confidential', 'display_judgenotes',
       'case_notecount', 'display_legalfileviewer', 'display_fileviewer',
       'can_userseepublicdocuments', 'can_userseecasedocuments',
       'can_userseeenoticehistory', 'can_selectdocket', 'can_seeecflinks',
       'can_seelegalfilelinks', 'is_ticket', 'address_a_type', 'address_city',
       'address_line_1', 'address_seq_no', 'address_state_code', 'address_zip',
       'birth_date', 'birth_date_code', 'criminal_case', 'criminal_ind',
       'description', 'description_code', 'first_name', 'year_of_birth',
       'formatted_party_address', 'formatted_party_name',
       'formatted_telephone', 'last_name', 'lit_ind', 'middle_name',
       'party_type', 'pidm', 'pred_code', 'prosecuting_atty', 'pty_seq_no',
       'sort_seq', 'age', 'case_desc', 'court_desc', 'location', 'filing_date',


In [67]:
# Fields
"""['case_id', 'court_id', 'participants', 'related_cases',
    'protection_order', 'parties', 'disposed', 'legal_fileaccepted',
    'paper_accepted', 'confidential', 'display_judgenotes',
    'case_notecount', 'display_legalfileviewer', 'display_fileviewer',
    'can_userseepublicdocuments', 'can_userseecasedocuments',
    'can_userseeenoticehistory', 'can_selectdocket', 'can_seeecflinks',
    'can_seelegalfilelinks', 'is_ticket', 'address_a_type', 'address_city',
    'address_line_1', 'address_seq_no', 'address_state_code', 'address_zip',
    'birth_date', 'birth_date_code', 'criminal_case', 'criminal_ind',
    'description', 'description_code', 'first_name', 'year_of_birth',
    'formatted_party_address', 'formatted_party_name',
    'formatted_telephone', 'last_name', 'lit_ind', 'middle_name',
    'party_type', 'pidm', 'pred_code', 'prosecuting_atty', 'pty_seq_no',
    'sort_seq', 'age', 'case_desc', 'court_desc', 'location', 'filing_date',
    'case_date', 'formatted_filingdate', 'case_type', 'case_security',
    'case_typecode', 'vine_code', 'locn_code', 'court_code', 'vine_display',
    'vine_id', 'dockets', 'documents', 'charges', 'judge', 'court_type',
    'ticket_searchresult', 'fine', 'plea_andpayind', 'ticket', 'ticket_img',
    'status', 'events', 'court_date', 'court_time', 'court_link',
    'arrest_date', 'arrest_time', 'where_held', 'gender', 'release_info',
    'source', 'custom']
"""

"['case_id', 'court_id', 'participants', 'related_cases',\n    'protection_order', 'parties', 'disposed', 'legal_fileaccepted',\n    'paper_accepted', 'confidential', 'display_judgenotes',\n    'case_notecount', 'display_legalfileviewer', 'display_fileviewer',\n    'can_userseepublicdocuments', 'can_userseecasedocuments',\n    'can_userseeenoticehistory', 'can_selectdocket', 'can_seeecflinks',\n    'can_seelegalfilelinks', 'is_ticket', 'address_a_type', 'address_city',\n    'address_line_1', 'address_seq_no', 'address_state_code', 'address_zip',\n    'birth_date', 'birth_date_code', 'criminal_case', 'criminal_ind',\n    'description', 'description_code', 'first_name', 'year_of_birth',\n    'formatted_party_address', 'formatted_party_name',\n    'formatted_telephone', 'last_name', 'lit_ind', 'middle_name',\n    'party_type', 'pidm', 'pred_code', 'prosecuting_atty', 'pty_seq_no',\n    'sort_seq', 'age', 'case_desc', 'court_desc', 'location', 'filing_date',\n    'case_date', 'formatted_fi

In [87]:
# Target fields first_name	last_name	street	suite	city	state	postal_code	mailing_street
cases_not_found_fields = df[
    [
        "case_id",
        "first_name",
        "last_name",
        "middle_name",
        "address_line_1",
        "address_city",
        "address_state_code",
        "address_zip",
        "formatted_party_address",
    ]
].rename(
    columns={
        "address_line_1": "street",
        "address_state_code": "state",
        "address_zip": "postal_code",
        "address_city": "city",
        "formatted_party_address": "mailing_street",
    }
)

In [88]:
# Remove the \n from the mailing_street
cases_not_found_fields.mailing_street = cases_not_found_fields.mailing_street.apply(
    lambda x: x.replace("\n", "") if isinstance(x, str) else x
)

In [89]:
cases_not_found_fields.to_csv("cases.csv", index=False)

In [90]:

def flatten(dictionary, parent_key="", separator="_"):
    items = []
    for key, value in dictionary.items():
        new_key = parent_key + separator + key if parent_key else key
        if isinstance(value, MutableMapping):
            items.extend(flatten(value, new_key, separator=separator).items())
        else:
            items.append((new_key, value))
    return dict(items)
